# Issue JSON reducer

issues.pickle has downloaded info from github api. kubernetes/kubernetes is a big resource of issues (about 84k  on 2019.11.03) but is huge for use case. In this notebook i will reduce it to make easier to process into PowerBI

In [ ]:
!pip install scalpl

In [ ]:
import pickle
from scalpl import Cut
import json
from tqdm.notebook import trange, tqdm

In [ ]:
with open('issues.pickle', 'rb') as handle:
    issues = pickle.load(handle)

In [ ]:
def clear_key_pattern(c_dict, key_pattern):
    # Proxy object to walk over it
    proxy_dict = Cut(c_dict)
    # Proxy object to operate data and return it
    data_dict = proxy_dict.copy()
    
    for i in proxy_dict.items():
        key = i[0]
        value = i[1]
        if key_pattern in key:
            #print(f"|{key}|{type(i[1])}")
            del data_dict[key]
            
        if type(value) is dict:
            data_dict[key] = clear_key_pattern(value, key_pattern)
            
        if type(value) is list:
            n_ele = []
            for c_value in value:
                n_ele.append(clear_key_pattern(c_value, key_pattern))
            data_dict[key] = n_ele
            
    return data_dict

In [ ]:
reduced_issues = {}

for i in tqdm(issues.items()):
    key = i[0]
    value = i[1]

    c_issue = clear_key_pattern(value, "url")
    c_issue = clear_key_pattern(c_issue, "node_id")
    c_issue = clear_key_pattern(c_issue, "body")
    reduced_issues[key] = c_issue

In [ ]:
with open("issues_list_reduced.json", "w") as handle:
    json.dump(list(reduced_issues.values()), handle)